In [2]:
#iniciar spark
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType

conf = SparkConf()
#conf.set('spark.jars.packages', 'org.apache.hadoop:hadoop-aws:3.2.2')
conf.set('spark.jars.packages', 'org.apache.hadoop:hadoop-aws:3.2.2,com.microsoft.azure:spark-mssql-connector_2.12:1.2.0')
conf.set('spark.hadoop.fs.s3a.aws.credentials.provider', 'com.amazonaws.auth.InstanceProfileCredentialsProvider')
spark = SparkSession.builder.config(conf=conf).getOrCreate()

arquivo_todos_csv = spark.read.option('delimiter', ',').option('header', 'true').csv('s3a://stc-grupo04-henrique-sptech-data-transform/preTratedCsv.csv/part-0000[0-3]-11b1c6d7-1ac5-4469-bdaa-adb68ed234ae-c000.csv')
arquivo_todos_json = spark.read.option('delimiter', ',').option('header', 'true').csv('s3a://stc-grupo04-henrique-sptech-data-transform/preTratedJson.csv/part-0000[0-4]-bc735acd-498d-4217-8f2f-29d5dbf13433-c000.csv')
arquivo_todos_txt = spark.read.option('delimiter', ',').option('header', 'true').csv('s3a://stc-grupo04-henrique-sptech-data-transform/preTratedTxt.csv/part-00000-13f5c5fb-d0de-412a-9471-1595218a4ff6-c000.csv')
print('arquivos lidos')

arquivos lidos


In [4]:
print(arquivo_todos_csv.count())
print(arquivo_todos_json.count())
print(arquivo_todos_txt.count())

1000000


1000001
28


In [3]:
arquivo_todos_csv.printSchema()
arquivo_todos_json.printSchema()
arquivo_todos_txt.printSchema()

root
 |-- ESTADO: string (nullable = true)
 |-- SEXO: string (nullable = true)
 |-- IDADE: string (nullable = true)
 |-- ETNIA: string (nullable = true)
 |-- EVOLUCAO_CASO: string (nullable = true)
 |-- PRIMEIRA_DOSE: string (nullable = true)
 |-- SEGUNDA_DOSE: string (nullable = true)

root
 |-- ESTADO: string (nullable = true)
 |-- SEXO: string (nullable = true)
 |-- IDADE: string (nullable = true)
 |-- ASMA: string (nullable = true)
 |-- CARDIOPATIA: string (nullable = true)
 |-- DIABETE: string (nullable = true)
 |-- OBESIDADE: string (nullable = true)
 |-- SIN_DOWN: string (nullable = true)

root
 |-- TERRITORIALIDADES: string (nullable = true)
 |-- MEDIA_ANOS_ESTUDOS: string (nullable = true)
 |-- INDICE_EDUCACAO: string (nullable = true)
 |-- LONGEVIDADE: string (nullable = true)
 |-- RENDA: string (nullable = true)
 |-- IDH: string (nullable = true)



In [5]:
#limpar registros em branco (se tiver mesmo)

from pyspark.sql.functions import concat

arquivo_semi_csv_1=(arquivo_todos_csv.select(
    col('ESTADO').alias('ESTADO_csv'),
    col('SEXO').alias('SEXO_csv'),
    col('IDADE').cast(IntegerType()).alias('IDADE_csv'),
    col('ETNIA'),
    col('EVOLUCAO_CASO'),
    col('PRIMEIRA_DOSE'),
    col('SEGUNDA_DOSE')
)).filter(col("ESTADO") != "").filter(col("SEXO")!= "").filter(col("IDADE")!= "") 

arquivo_semi_json=arquivo_todos_json.filter(col("ESTADO") != "").filter(col("SEXO")!= "").filter(col("IDADE")!= "")

print(arquivo_semi_csv_1.count())
print(arquivo_semi_json.count())

970888


936522


In [6]:
#Criando identificador único para realizar a junção das bases (CSV e JSON)

from pyspark.sql.window import *
from pyspark.sql.functions import row_number

arquivo_semi_csv_2=arquivo_semi_csv_1.withColumn("row_num", row_number().over(Window.partitionBy("ESTADO_csv","SEXO_csv","IDADE_csv").orderBy("IDADE_csv")))
arquivo_semi_json_2=arquivo_semi_json.withColumn("row_num", row_number().over(Window.partitionBy("ESTADO","SEXO","IDADE").orderBy("IDADE")))

In [7]:
from pyspark.sql.functions import upper 

join_csv_json = arquivo_semi_json_2.join(arquivo_semi_csv_2,((arquivo_semi_json_2.ESTADO==arquivo_semi_csv_2.ESTADO_csv) & 
                                                          (arquivo_semi_json_2.SEXO==upper(arquivo_semi_csv_2.SEXO_csv)) &
                                                          (arquivo_semi_json_2.IDADE==arquivo_semi_csv_2.IDADE_csv) &
                                                          (arquivo_semi_json_2.row_num==arquivo_semi_csv_2.row_num)
                                                          ),"inner")



In [8]:
join_csv_json.printSchema()

root
 |-- ESTADO: string (nullable = true)
 |-- SEXO: string (nullable = true)
 |-- IDADE: string (nullable = true)
 |-- ASMA: string (nullable = true)
 |-- CARDIOPATIA: string (nullable = true)
 |-- DIABETE: string (nullable = true)
 |-- OBESIDADE: string (nullable = true)
 |-- SIN_DOWN: string (nullable = true)
 |-- row_num: integer (nullable = false)
 |-- ESTADO_csv: string (nullable = true)
 |-- SEXO_csv: string (nullable = true)
 |-- IDADE_csv: integer (nullable = true)
 |-- ETNIA: string (nullable = true)
 |-- EVOLUCAO_CASO: string (nullable = true)
 |-- PRIMEIRA_DOSE: string (nullable = true)
 |-- SEGUNDA_DOSE: string (nullable = true)
 |-- row_num: integer (nullable = false)



In [9]:
join_csv_json.show(10)

+------+---------+-----+--------+-----------+--------+---------+--------+-------+----------+---------+---------+--------+-------------+-------------------+-------------------+-------+
|ESTADO|     SEXO|IDADE|    ASMA|CARDIOPATIA| DIABETE|OBESIDADE|SIN_DOWN|row_num|ESTADO_csv| SEXO_csv|IDADE_csv|   ETNIA|EVOLUCAO_CASO|      PRIMEIRA_DOSE|       SEGUNDA_DOSE|row_num|
+------+---------+-----+--------+-----------+--------+---------+--------+-------+----------+---------+---------+--------+-------------+-------------------+-------------------+-------+
|    AC| FEMININO|   27|IGNORADO|   IGNORADO|IGNORADO| IGNORADO|IGNORADO|      1|        AC| Feminino|       27|Ignorado|         null|   SINOVAC/BUTANTAN|   SINOVAC/BUTANTAN|      1|
|    AC| FEMININO|   28|IGNORADO|   IGNORADO|IGNORADO| IGNORADO|IGNORADO|      1|        AC| Feminino|       28|  Branca|         Cura|ASTRAZENECA/FIOCRUZ|ASTRAZENECA/FIOCRUZ|      1|
|    AC| FEMININO|   31|IGNORADO|   IGNORADO|IGNORADO| IGNORADO|IGNORADO|      2

In [10]:
#limpando colunas ESTADO SEXO IDADE duplicadas
#Tabela 1 pronta
join_csv_json_final = join_csv_json.select(
    col('ESTADO'),
    col('SEXO'),
    col('IDADE').cast(IntegerType()),
    col('ETNIA'),
    col('ASMA'),
    col('CARDIOPATIA'),
    col('DIABETE'),
    col('SIN_DOWN'),
    col('EVOLUCAO_CASO'),
    col('PRIMEIRA_DOSE'),
    col('SEGUNDA_DOSE')
).limit(1000000)

In [31]:
type(join_csv_json_final)
print(join_csv_json_final.count())

215880



[Stage 102:============================>                            (1 + 1) / 2]



In [12]:
#Criando tabela 2
#Fazendo o group by na base concatenada(join_csv_json_final) pelos meses
from pyspark.sql.functions import first
from pyspark.sql.functions import avg 
from pyspark.sql.functions import when 
from pyspark.sql.functions import sum as _sum
from pyspark.sql.functions import max as _max
from pyspark.sql.functions import lit
from pyspark.sql.functions import count as _count
from pyspark.sql.functions import round as _round


#criando colunas de sexo p/ futuro join

df_pivot_sexo=join_csv_json_final.select(
    col('ESTADO').alias('ESTADO_sex'),
    col('IDADE').cast(IntegerType()),
    col('SEXO')
).groupBy("ESTADO_sex").pivot("SEXO").avg("IDADE")


In [13]:
#Somando doenças por estado

df_doenca = join_csv_json_final.select(
    col('ESTADO').alias('ESTADO_doe'),
    when(col('ASMA')=="SIM",1).otherwise(0).alias('asma_trat'),
    when(col('CARDIOPATIA')=="SIM",1).otherwise(0).alias('card_trat'),
    when(col('DIABETE')=="SIM",1).otherwise(0).alias('diab_trat'),
    when(col('SIN_DOWN')=="SIM",1).otherwise(0).alias('down_trat')
).groupBy('ESTADO_doe').agg(_sum('asma_trat'),_sum('card_trat'),_sum('diab_trat'),_sum('down_trat'))


In [30]:
#Somando evolução dos casos

#agrupando
df_pivot_evolucao_1=join_csv_json_final.select(
    col('ESTADO'),
    when(col('EVOLUCAO_CASO')=="null","ignorado").
    when(col('EVOLUCAO_CASO')=="Ignorado","ignorado").
    when(col('EVOLUCAO_CASO')=="Cancelado","ignorado").
    when(col('EVOLUCAO_CASO')=="Cura","cura").
    when(col('EVOLUCAO_CASO').like('Em tratamento%'),"cura").
    when(col('EVOLUCAO_CASO')=="Óbito","obito").
    when(col('EVOLUCAO_CASO').like("Internado%"),"internado").
    otherwise("ignorado").
    alias("EVOLUCAO_CASO")
).withColumn("contagem",lit(1))

#somando
df_pivot_evolucao_2=df_pivot_evolucao_1.select(
    col('ESTADO').alias('ESTADO_evo'),
    col('EVOLUCAO_CASO'),
    col('contagem')
).groupBy("ESTADO_evo").pivot("EVOLUCAO_CASO").sum('contagem')

#Removendo valores nulos da contagem
df_pivot_evolucao_3=df_pivot_evolucao_2.select(
    col('ESTADO_evo'),
    when(((col('cura').cast("int").isNull())) ,0).otherwise(col('cura')).alias('cura'),
    when(((col('ignorado').cast("int").isNull())) ,0).otherwise(col('ignorado')).alias('ignorado'),
    when(((col('internado').cast("int").isNull())) ,0).otherwise(col('internado')).alias('internado'),
    when(((col('obito').cast("int").isNull())) ,0).otherwise(col('obito')).alias('obito')
)

+----------+----+--------+---------+-----+
|ESTADO_evo|cura|ignorado|internado|obito|
+----------+----+--------+---------+-----+
|        AC|   6|      16|        0|    0|
|        AL|  29|      85|        0|    0|
|        AM|  81|     172|        0|    0|
|        AP|   2|      23|        0|    0|
|        BA| 129|     772|        0|    0|
|        CE| 120|     457|        0|    0|
|        DF|  31|     574|        0|    0|
|        ES|   4|     297|        0|    0|
|        GO| 127|     424|        0|    0|
|        MA|   7|     172|        0|    0|
|        MG| 494|    1925|        0|    1|
|        MS| 246|     145|        0|    0|
|        MT|  42|     254|        0|    0|
|        PA|  14|     225|        0|    0|
|        PB| 102|     163|        0|    1|
|        PE| 318|     596|        0|    0|
|        PI|  12|     125|        0|    0|
|        PR|  70|    1234|        0|    0|
|        RJ|  98|    1780|        0|    0|
|        RN|   2|     273|        0|    0|
+----------

In [27]:
#Transformando a coluna TERRITORIALIDADES em siglas

df_idh=arquivo_todos_txt.withColumn('SIGLA',
                            when(col('TERRITORIALIDADES')=='São Paulo',"SP").
                            when(col('TERRITORIALIDADES')=='Brasil',"BR").
                            when(col('TERRITORIALIDADES')=='Acre',"AC").
                            when(col('TERRITORIALIDADES')=='Alagoas',"AL").
                            when(col('TERRITORIALIDADES')=='Amapá',"AP").
                            when(col('TERRITORIALIDADES')=='Amazonas',"AM").
                            when(col('TERRITORIALIDADES')=='Bahia',"BA").
                            when(col('TERRITORIALIDADES')=='Ceará',"CE").
                            when(col('TERRITORIALIDADES')=='Distrito Federal',"DF").
                            when(col('TERRITORIALIDADES')=='Espírito Santo',"ES").
                            when(col('TERRITORIALIDADES')=='Goiás',"GO").
                            when(col('TERRITORIALIDADES')=='Maranhão',"MA").
                            when(col('TERRITORIALIDADES')=='Mato Grosso do Sul',"MS").
                            when(col('TERRITORIALIDADES')=='Mato Grosso',"MT").
                            when(col('TERRITORIALIDADES')=='Minas Gerais',"MG").
                            when(col('TERRITORIALIDADES')=='Paraíba',"PB").
                            when(col('TERRITORIALIDADES')=='Paraná',"PR").
                            when(col('TERRITORIALIDADES')=='Pará',"PA").
                            when(col('TERRITORIALIDADES')=='Pernambuco',"PE").
                            when(col('TERRITORIALIDADES')=='Piauí',"PI").
                            when(col('TERRITORIALIDADES')=='Rio de Janeiro',"RJ").
                            when(col('TERRITORIALIDADES')=='Rio Grande do Norte',"RN").
                            when(col('TERRITORIALIDADES')=='Rio Grande do Sul',"RS").
                            when(col('TERRITORIALIDADES')=='Rondônia',"RO").
                            when(col('TERRITORIALIDADES')=='Roraima',"RR").
                            when(col('TERRITORIALIDADES')=='Santa Catarina',"SC").
                            when(col('TERRITORIALIDADES')=='Sergipe',"SE").
                            when(col('TERRITORIALIDADES')=='Tocantins',"TO")                             
                            )

In [31]:
#JOIN nas tabelas tratadas anteriormente

df_estado_join_1 = df_pivot_sexo.join(df_doenca,
                                    df_pivot_sexo.ESTADO_sex==df_doenca.ESTADO_doe,
                                    "inner")

df_estado_join_2 = df_estado_join_1.join(df_pivot_evolucao_3,
                                           df_pivot_evolucao_3.ESTADO_evo==df_estado_join_1.ESTADO_sex,
                                           "inner")

df_estado_join_3 = df_estado_join_2.join(df_idh,
                                        df_estado_join_2.ESTADO_sex==df_idh.SIGLA,
                                        "inner")




df_estado_join_final = df_estado_join_3.select(
    col('ESTADO_sex').alias('estado'),
    col('IDH').alias('idh'),
    col('MEDIA_ANOS_ESTUDOS').alias('media_anos_estudo'),
    col('INDICE_EDUCACAO').alias('indice_educacao'),
    col('LONGEVIDADE').alias('longevidade'),
    col('RENDA').alias('renda'),
    _round(col('FEMININO')).alias('media_idade_feminina'),
    _round(col('MASCULINO')).alias('media_idade_masculina'),
    col('sum(asma_trat)').alias('qte_asma'),
    col('sum(card_trat)').alias('qte_cardiopatia'),
    col('sum(diab_trat)').alias('qte_diabete'),
    col('sum(down_trat)').alias('qte_down'),
    col('cura').alias('casos_curados'),
    col('ignorado').alias('casos_ignorados'),
    col('internado').alias('casos_internados'),
    col('obito').alias('casos_obitos')
)

In [32]:
#Salvando S3 casos e evolução
join_csv_json_final.write.mode('overwrite').option("header", "true").csv('s3a://stc-grupo04-henrique-sptech-data-final/trated_tabel_csv_json.csv')

#Salvando S3 estado_IDH
df_estado_join_final.write.mode('overwrite').option("header", "true").csv('s3a://stc-grupo04-henrique-sptech-data-final/trated_tabel_inner_txt.csv/')

22/06/04 19:07:13 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
22/06/04 19:07:30 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
22/06/04 19:07:33 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
22/06/04 19:08:08 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.


In [35]:
#Salvando Azure - casos e evolução
print("inserindo azure")
join_csv_json_final.write \
.format('com.microsoft.sqlserver.jdbc.spark') \
.mode('overwrite') \
.option('driver', 'com.microsoft.sqlserver.jdbc.SQLServerDriver') \
.option('url', 'jdbc:sqlserver://servidor-grupo04.database.windows.net;databaseName=grupo04;') \
.option('dbtable', 'table_casos_doencas') \
.option('user', 'urubu100') \
.option('password', 'Urubu1@@') \
.save()
print("Tabela por caso salva")

#Salvando Azure - consolidado estado
print("inserindo azure")
df_estado_join_final.write \
.format('com.microsoft.sqlserver.jdbc.spark') \
.mode('overwrite') \
.option('driver', 'com.microsoft.sqlserver.jdbc.SQLServerDriver') \
.option('url', 'jdbc:sqlserver://servidor-grupo04.database.windows.net;databaseName=grupo04;') \
.option('dbtable', 'table_estado_evolucao') \
.option('user', 'urubu100') \
.option('password', 'Urubu1@@') \
.save()
print("Tabela por estado salva")

inserindo azure


Tabela por caso salva
inserindo azure


Tabela por estado salva
